# Visual Assessment

Slice plot for
* 1 slice - 1 models
* 1 slice - N models (sum + subtraction)

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import cv2
import copy
import time
import ast
import json
import tensorflow as tf

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from plotly.colors import n_colors

In [ ]:
import sys
import os

sys.path.append(os.path.abspath('../data_utils'))
sys.path.append(os.path.abspath('../models'))

In [ ]:
from model_zoo.utils import check_gpu
from model_zoo.losses.dice import DiceLoss, DiceCoefficient
from data_utils.DataContainer import DataContainer
from data_utils.TestSet import TestSet

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

# TestSet

In [ ]:
MODELS_SIMPLE1 = ["XNet_T2_relu", "XNet_T2_leaky", "XNet_T2_selu"]
MODELS_SIMPLE2 = ["XNet_T1_relu", "XNet_T1_leaky", "XNet_T1_selu"]
MODELS_SIMPLE = [*MODELS_SIMPLE2, *MODELS_SIMPLE1]
MODELS_CG = ["CG_XNet_T1_relu", "CG_XNet_T2_relu"]
MODELS_DA = ["SegmS2T_GAN1_relu", "SegmS2T_GAN2_relu", "SegmS2T_GAN5_relu"]
MODELS_GAN = ["GAN_1+XNet_T1_relu", "GAN_2+XNet_T1_relu", "GAN_5+XNet_T1_relu"]
MODELS = [*MODELS_SIMPLE, *MODELS_CG, *MODELS_GAN, *MODELS_DA]
MODELS_BASELINE = [*MODELS_SIMPLE, *MODELS_CG]
MODELS_DA = [*MODELS_DA, *MODELS_GAN]

In [ ]:
testset = TestSet("/tf/workdir/data/VS_segm/VS_registered/test_processed/", load=True)

In [ ]:
df_total = testset.df_total
intermediate = df_total.to_json()
df_total = pd.read_json(intermediate)

In [ ]:
df_total

# First 1 slice - 1 method plot

In [ ]:
patient_id = 200
slide = 32
model = "XNet_T2_relu"

In [ ]:
df = pd.read_json(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/evaluation.json")
container = DataContainer(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/")
img = container.t2_scan_slice(slide)
segm = cv2.drawContours(np.zeros((256,256)), np.array(df.iloc[slide][f"VS_segm_pred-{model}"]), -1, (1), 1)
gt = cv2.drawContours(np.zeros((256,256)), np.array(df.iloc[slide][f"VS_segm_gt"]), -1, (1), -1)

In [ ]:
df_2d = pd.DataFrame(columns=["slice", "gt", "pred", "img", "class"])
df_2d = df_2d.append({"slice": slide, "img": img, "gt": gt, 
                      "pred": segm, "class": df.iloc[slide][f"VS_class_pred-{model}"]},
                    ignore_index=True)

In [ ]:
colorscale_segm = [[0, 'rgba(0,0,0,0)'],
                 [0.99, 'rgba(0,0,0,0)'],
                 [1.0, 'rgba(255,0,0,1)']]  # 165,0,38,1 - RdYlBu[0]
colorscale_gt = [[0, 'rgba(0,0,0,0)'],
                 [0.99, 'rgba(0,0,0,0)'],
                 [1, 'rgba(0,0,0,1)']] # 49,54,149,1 - RdYlBu[-1], rgba(240,249,33,1)

In [ ]:
fig_total = make_subplots(1,2, column_widths=[0.9,0.1], horizontal_spacing=0.05)
fig = go.Figure() # make_subplots(1,2)
fig_img = px.imshow(img, binary_string=True)
fig.add_trace(fig_img.data[0]) # ,1,1)
fig.update_traces(opacity=1.0)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.add_heatmap(z=gt, hoverinfo="skip", showscale=False, colorscale=colorscale_gt, 
                opacity=0.8, name="gt") # ,row=1,col=1)
fig.add_heatmap(z=segm, hoverinfo="skip", showscale=False, colorscale=colorscale_segm, 
                opacity=0.5, name="segm") # ,row=1, col=1)
fig.update_layout(margin=dict(l=5,
                              r=5,
                              b=5,
                              t=5,
                              pad=4)
                  )
#fig.update_traces(colorbar=dict(visible=False), selector=dict(type='heatmap'))
#fig.update_traces(colorbar=dict(
#    title="Contours", tickmode="array", ticktext =["gt", "pred"], tickvals = [0.9,1]), selector=dict(type='heatmap'))                                     

# First 1 slice - N Methods - Sum

In [ ]:
patient_id = 241
slide = 29
model = ["XNet_T2_relu", "XNet_T2_leaky", "XNet_T2_selu", "CG_XNet_T2_relu","XNet_T2_leaky", "XNet_T2_selu", "CG_XNet_T2_relu"]

In [ ]:
df = pd.read_json(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/evaluation.json")
container = DataContainer(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/")
img = container.t2_scan_slice(slide)
gt = cv2.drawContours(np.zeros((256,256)), np.array(df.iloc[slide][f"VS_segm_gt"]), -1, (1), 1)
segmentation = []
for m in model:
    segm = cv2.drawContours(np.zeros((256,256)), [np.array(s).astype(np.int64) for s in np.array(df.iloc[slide][f"VS_segm_pred-{m}"], dtype="object")], -1, (1), -1)
    segmentation.append(segm)
segm_sum = (np.sum(segmentation, axis=0))
z_max = len(segmentation)

In [ ]:
bvals = list(range(0, z_max+1,1))
nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals] 

red = n_colors('rgba(253,219,199)', 'rgba(103,0,31)', z_max, colortype='rgb')

colors = red
colors = [c.replace("rgb","rgba").replace(")",",1)") if "rgba" not in c else c for c in colors]

dcolorscale = [] #discrete colorscale
for k in range(len(colors)):
    dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])

In [ ]:
bvals2 = np.array(bvals)
if len(model) < 5:
    tickvals = np.linspace(-len(segmentation)+1,len(segmentation)-1,len(colors)).tolist() #[np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] # 
elif len(model) >= 5:
    tickvals = np.linspace(-len(segmentation)+1,len(segmentation)-1,len(colors)).tolist()
ticktext = [f"{k}"for k in bvals2[1:]]
tickvals, ticktext

In [ ]:
segm = segm_sum
segm[segm == 0] = None
hovertext = list()
for yi, yy in enumerate(segm):
    hovertext.append(list())
    for xi, xx in enumerate(segm[0]):
        hovertext[-1].append('value: {}'.format(segm[yi][xi]))

In [ ]:
fig_total = make_subplots(1,2, column_widths=[0.9,0.1], horizontal_spacing=0.05)
fig = go.Figure() 
fig_img = px.imshow(img, binary_string=True)
fig.add_trace(fig_img.data[0]) 
fig.update_traces(hovertemplate=None, hoverinfo="skip")
fig.update_traces(opacity=1.0)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.add_heatmap(z=segm, showscale=True, colorscale=dcolorscale, 
                zmid=0, hoverinfo='text', text=hovertext, hoverongaps=False,
                colorbar = dict(thickness=30, tickvals=tickvals, ticktext=ticktext),
                opacity=0.9, name="segm")
fig.update_layout(margin=dict(l=5, r=2, b=5, t=5, pad=4))

# Second 1 slice - N models - Subtraction

In [ ]:
patient_id = 246
slide = 38
model = ["XNet_T2_relu", "XNet_T2_leaky", "XNet_T2_selu", "CG_XNet_T2_relu", "CG_XNet_T1_relu", "XNet_T2_relu",
        "XNet_T2_relu"]
len(model)

In [ ]:
df = pd.read_json(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/evaluation.json")
container = DataContainer(f"/tf/workdir/data/VS_segm/VS_registered/test_processed/vs_gk_{patient_id}/")
img = container.t2_scan_slice(slide)
segmentation = []
for m in model:
    segm = cv2.drawContours(np.zeros((256,256)), [np.array(s) for s in np.array(df.iloc[slide][f"VS_segm_pred-{m}"])], -1, (1), -1)
    segmentation.append(segm)
segm_sum = (np.sum(segmentation, axis=0))
gt = cv2.drawContours(np.zeros((256,256)), np.array(df.iloc[slide][f"VS_segm_gt"]), -1, (len(segmentation)), -1)
segm_subtract = gt-segm_sum
z_max = len(segmentation)

In [ ]:
bvals = list(range(-z_max-1, z_max+1,1))
nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals] 

red = n_colors('rgba(253,219,199)', 'rgba(103,0,31)', z_max, colortype='rgb')
blue = n_colors('rgb(5,48,97)', 'rgb(209,229,240)', z_max, colortype='rgb')

colors = blue + ['rgba(255,255,255,0)'] + red
colors = [c.replace("rgb","rgba").replace(")",",1)") if "rgba" not in c else c for c in colors]

dcolorscale = [] #discrete colorscale
for k in range(len(colors)):
    dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])

In [ ]:
bvals2 = np.array(bvals)
if len(model) < 5:
    tickvals = np.linspace(-len(segmentation)+0.5,len(segmentation)-0.5,len(colors)).tolist() #[np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] # 
elif len(model) >= 5:
    tickvals = np.linspace(-len(segmentation)+1.5,len(segmentation)-1.5,len(colors)).tolist()
ticktext = [f"{k}"for k in bvals2[1:]]

In [ ]:
segm_subtract[segm_subtract == 0] = None
hovertext = list()
for yi, yy in enumerate(segm_subtract):
    hovertext.append(list())
    for xi, xx in enumerate(segm_subtract[0]):
        hovertext[-1].append('value: {}'.format(segm_subtract[yi][xi]))

In [ ]:
fig_total = make_subplots(1,2, column_widths=[0.9,0.1], horizontal_spacing=0.05)
fig = go.Figure() 
fig_img = px.imshow(img, binary_string=True)
fig.add_trace(fig_img.data[0]) 
fig.update_traces(hovertemplate=None, hoverinfo="skip")
fig.update_traces(opacity=1.0)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.add_heatmap(z=segm_subtract, showscale=True, colorscale=dcolorscale, 
                zmid=0, hoverinfo='text', text=hovertext, hoverongaps=False,
                colorbar = dict(thickness=30, tickvals=tickvals, ticktext=ticktext),
                opacity=0.8, name="segm")
fig.update_layout(margin=dict(l=5, r=2, b=5, t=5, pad=4))